In [5]:
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io

display(HTML("""
<style>
  .custom-button {
    background: #007BFF !important;
    border-radius: 8px !important;
    color: white !important;
    font-weight: bold !important;
    font-size: 16px !important;
    padding: 15px 30px !important;
    margin: 10px !important;
    border: none !important;
    transition: 0.3s ease-in-out !important;
  }
  .custom-button:hover {
    background: #0056b3 !important;
    transform: scale(1.05);
  }
  .header {
    color: #212529;
    font-family: 'Arial';
    text-align: center;
    font-size: 2.8em;
    font-weight: bold;
    margin: 30px 0;
  }
  .output-area {
    border-radius: 10px;
    padding: 20px;
    margin: 15px 0;
    background: #f1f3f5;
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
  }
</style>
"""))

header = widgets.HTML("<h1 class='header'>Image Steganography Tool</h1>")
upload_cover = widgets.FileUpload(description="Upload Cover Image", multiple=False)
upload_secret = widgets.FileUpload(description="Upload Secret Image", multiple=False)
process_btn = widgets.Button(description="Encode Image", layout=widgets.Layout(width='300px', height='50px'))
show_encoded_btn = widgets.Button(description="Show Encoded Image", layout=widgets.Layout(width='300px', height='50px'))
show_hidden_btn = widgets.Button(description="Reveal Hidden Image", layout=widgets.Layout(width='300px', height='50px'))

for btn in [process_btn, show_encoded_btn, show_hidden_btn]:
    btn.add_class('custom-button')

output = widgets.Output()
image_display = widgets.Output()

buttons_box = widgets.HBox([show_encoded_btn, show_hidden_btn], layout=widgets.Layout(justify_content='center'))
main_box = widgets.VBox([
    header,
    upload_cover,
    upload_secret,
    process_btn,
    buttons_box,
    image_display
], layout=widgets.Layout(align_items='center', width='80%', padding='20px'))

def encode_images(b):
    global encoded_image, decoded_image
    with output:
        clear_output()
        try:
            if not upload_cover.value or not upload_secret.value:
                raise ValueError("Please upload both cover and secret images!")

            cover_data = next(iter(upload_cover.value.values()))['content']
            secret_data = next(iter(upload_secret.value.values()))['content']

            cover = Image.open(io.BytesIO(cover_data)).convert("RGB")
            secret = Image.open(io.BytesIO(secret_data)).convert("RGB")

            secret = secret.resize(cover.size)

            cover_arr = np.array(cover)
            secret_arr = np.array(secret)

            cover_arr &= 0b11110000
            secret_bits = (secret_arr >> 4).astype(np.uint8)
            encoded_arr = cover_arr | secret_bits

            global encoded_image, decoded_image
            encoded_image = Image.fromarray(encoded_arr)
            decoded_image = decode_image(encoded_image)

            display(HTML("<h3 class='output-area'>Encoding Successful!</h3>"))
            add_download_buttons()
        except Exception as e:
            display(HTML(f"<h3 class='output-area' style='color:red;'>Error: {str(e)}</h3>"))

def decode_image(encoded_img):
    encoded_arr = np.array(encoded_img)
    secret_bits = (encoded_arr & 0b00001111).astype(np.uint8)
    return Image.fromarray(secret_bits << 4)

def show_encoded(b):
    with image_display:
        clear_output()
        if encoded_image:
            plt.figure(figsize=(10, 8))
            plt.imshow(encoded_image)
            plt.title("Encoded Image", fontsize=16)
            plt.axis('off')
            plt.show()
        else:
            display(HTML("<p class='output-area' style='color:red;'>Encode images first!</p>"))

def show_hidden(b):
    with image_display:
        clear_output()
        if decoded_image:
            plt.figure(figsize=(10, 8))
            plt.imshow(decoded_image)
            plt.title("Revealed Secret Image", fontsize=16)
            plt.axis('off')
            plt.show()
        else:
            display(HTML("<p class='output-area' style='color:red;'>Encode images first!</p>"))

def add_download_buttons():
    with image_display:
        clear_output()
        encoded_download_btn = widgets.Button(description="Download Encoded Image", layout=widgets.Layout(width='300px', height='50px'))
        secret_download_btn = widgets.Button(description="Download Secret Image", layout=widgets.Layout(width='300px', height='50px'))

        def on_encoded_download_click(b):
            encoded_image.save("encoded_image.png")
            files.download("encoded_image.png")

        def on_secret_download_click(b):
            decoded_image.save("decoded_secret.png")
            files.download("decoded_secret.png")

        encoded_download_btn.on_click(on_encoded_download_click)
        secret_download_btn.on_click(on_secret_download_click)
        display(widgets.HBox([encoded_download_btn, secret_download_btn], layout=widgets.Layout(justify_content='center')))

process_btn.on_click(encode_images)
show_encoded_btn.on_click(show_encoded)
show_hidden_btn.on_click(show_hidden)

display(main_box, output)


Output()

Image Steganography App

Name : Mahnoor Awan

Reg ID : 221858

Submitted to : Sir Ghulam Ali

Subject : DIP

This Notebook application is an **Image Steganography Tool** that hides a secret image inside a cover image using **least significant bit (LSB) encoding**.  

### **How It Works**  
1. **User Uploads Images**  
   - A cover image (carrier) and a secret image (hidden content) are uploaded.  
   - The secret image is resized to match the cover image.  

2. **Encoding Process**  
   - The least significant 4 bits of each pixel in the cover image are replaced with the most significant 4 bits of the secret image.  
   - The result is an encoded image that looks like the cover but contains the hidden image inside.  

3. **Displaying the Encoded Image**  
   - Users can view the encoded image to see if the secret is hidden properly.  

4. **Decoding Process**  
   - Extracts the hidden image by shifting the least significant 4 bits of the encoded image to the left, restoring the secret image.  

5. **Download Options**  
   - Users can download both the encoded and extracted secret images.  

**Git-hub project Link :**

**LinkedIn Post Link :** https://www.linkedin.com/posts/mahnoor-awan-75b351343_machinelearning-imageprocessing-airuniversity-activity-7310358604233019407-om5o?utm_source=share&utm_medium=member_desktop&rcm=ACoAAFYDs_cBz6cnw77pN1sd3OIVDzifBoS8eDM

